# Library

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as ds
import torchvision.transforms as transforms

import lightning as L
from lightning.pytorch.trainer import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Learning Rate Scheduler

딥러닝 모델을 훈련 시, learning rate를 조정하는 기법. <br>
learning rate는 네트워크의 가중치를 얼마나 크게 업데이트할지를 결정하는 하이퍼파라미터로, <br>
적절한 학습률을 설정하는 것은 모델이 빠르고 안정적으로 수렴하는 데 매우 중요. <br>

learning rate scheduler는 훈련 과정에서 학습률을 동적으로 조정하여 모델 성능을 개선하는 데 기여 <br>
- Step Decay: 학습률을 일정한 에포크마다 고정된 비율로 감소
- Exponential Decay: 학습률을 지수 함수에 따라 점진적으로 감소
- Cyclic Learning Rate: 학습률을 주기적으로 증가와 감소를 반복
- Plateau: 검증 손실이 개선되지 않는 경우 학습률을 감소
- Combination: 다양한 스케줄러를 조합

<br>

<span style="font-size: 20pt;"> 파이썬 코드 </span>

configure_optimizers에 scheduler를 추가. <br>
return에 scheduler 객체 return 추가. <br>

> ```python
> def configure_optimizers(self):
>     optimizer = optim.Adam(
>         self.model.parameter(),
>         lr=self.learning_rate,
>     )
>     scheduler = lr_scheduler.OneCycleLR(
>         optimizer,
>         max_lr=1e-3,
>         total_steps=self.trainer.estimated_stepping_batches,
>     )
>     return optimizer, scheduler
> ````

## StepLR

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb2BFko%2FbtqOUlVgZU3%2FOA2OWctZ3wF2Eakhpaf1Y1%2Fimg.png)

learning rate를 step_size(epoch)마다 감마로 감쇠. <br>

<br>

<span style="font-size: 16pt;"> 파이썬 코드 </span>

> ```python
> scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
> ````

<br>

<span style="font-size: 14pt;"> Paramter </span>

- step_size (int) – 어느 주기로 learning rate를 감소시킬 건지 설정
- gamma (float) – 감소될 학습률 (default: 0.1)

## ExponentialLR

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F7kmnL%2FbtqOMp5Xu77%2FE8Jk5ydKXYsw71nZcSN831%2Fimg.png)

매 epoch마다 이전 learning rate에 gamma만큼 곱하여 감쇄

<br>

<span style="font-size: 16pt;"> 파이썬 코드 </span>

> ```python
> scheduler = ExponentialLR(optimizer, gamma=0.1)
> ````

<br>

<span style="font-size: 14pt;"> Paramter </span>

- gamma (float) – 감소될 학습률 (default: 0.1)

## CyclicLR

|triangular|triangular2|exp_range|
|----------|-----------|---------|
|![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FVwbCj%2FbtqON8iY9Fp%2FVNODkZFUIbtG3I0wEkEiK1%2Fimg.png)|![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FRb1s0%2FbtqO0glZvNG%2FwHB48CykF7jt4zkzk7ROSk%2Fimg.png)|![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbtIC6z%2FbtqON9vvgpt%2FL5qQdBWHFlzg6GU9Bh2vk1%2Fimg.png)|

일정 주기로 설정한 최솟값과 최댓값 사이의 learning rate를 갖도록 함 <br>
learning rate를 매 배치 후 변경 <br>
- triangular: 선형적으로 증가하고 감소
- triangular2: 학습률이 선형적으로 증가하고 감소하지만, 사이클이 진행됨에 따라 최댓값이 줄어듦
- exp_range: 지수적으로 증가하고 감소

<br>

<span style="font-size: 16pt;"> 파이썬 코드 </span>

> ```python
> scheduler = CyclicLR(optimizer, base_lr, max_lr, step_size_up=2000, step_size_down=None, mode='triangular', gamma=1.0)
> ````

<br>

<span style="font-size: 14pt;"> Paramter </span>

- base_lr: 사이클의 최소 learning rate
- max_lr: 사이클의 최대 learning rate
- step_size_up: learning rate가 base_lr에서 max_lr로 증가하는 데 필요한 단계 수
- step_size_down: learning rate가 max_lr에서 다시 base_lr로 감소하는 데 필요한 단계 수.
- mode: 사이클의 모양을 지정하는 매개변수
    - triangular, triangular2, exp_range 중 설정

## ReduceLROnPlateau

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb25iNZ%2FbtqOKXvn072%2FawCat07o8yb9MHLRHeFUek%2Fimg.png)

모델의 성능이 개선되지 않을 때 학습률을 자동으로 감소시켜 학습을 안정화하고 성능을 향상시키기 위한 방법

<br>

<span style="font-size: 16pt;"> 파이썬 코드 </span>

> ```python
> scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, min_lr=0)
> ````

<br>

<span style="font-size: 14pt;"> Paramter </span>

- mode: 모니터링할 metric의 방향을 설정
    - min은 메트릭이 최소일 때, max는 메트릭이 최대일 때 학습률 감소
- factor: 학습률 감소 비율로 factor가 0.1이라면 이전 learning rate에 0.1을 곱함
- patience: 메트릭이 개선되지 않을 때까지 기다리는 에포크 수
- min_lr: 최소 learning rate

In [3]:
def convert_category_into_integer(df: pd.DataFrame, columns: list):
    """
    주어진 DataFrame의 특정 열들을 범주형에서 정수형으로 변환합니다.
    
    Parameters:
    - df (pd.DataFrame): 변환할 데이터프레임
    - columns (list): 범주형에서 정수형으로 변환할 열 이름의 리스트
    
    Returns:
    - pd.DataFrame: 변환된 데이터프레임
    - dict: 각 열에 대해 적합한 LabelEncoder 객체를 포함하는 딕셔너리
    """
    label_encoders = {}  # 각 열의 LabelEncoder 객체를 저장할 딕셔너리입니다.
    
    for column in columns:
        # 각 열에 대해 LabelEncoder 객체를 생성합니다.
        label_encoder = LabelEncoder()
        
        # LabelEncoder를 사용하여 해당 열의 범주형 데이터를 정수형으로 변환합니다.
        df.loc[:, column] = label_encoder.fit_transform(df[column])
        
        # 변환된 LabelEncoder 객체를 딕셔너리에 저장합니다.
        label_encoders.update({column: label_encoder})
    
    # 변환된 데이터프레임과 LabelEncoder 객체를 포함하는 딕셔너리를 반환합니다.
    return df, label_encoders

titanic = sns.load_dataset('titanic')
titanic.deck = titanic.deck.astype(str)
titanic = titanic.dropna()

titanic, _ = convert_category_into_integer(titanic, ('sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive'))

titanic = titanic.astype(np.float32)

train, temp = train_test_split(titanic, test_size=0.4, random_state=seed)

# 임시 데이터를 검증용 데이터와 테스트용 데이터로 분할
# 임시 데이터의 절반을 검증용 데이터로, 나머지 절반을 테스트용 데이터로 사용
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)



class TitanicDataset(Dataset):
    def __init__(self, data):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.data = data  # 데이터프레임을 저장합니다.
    
    def __len__(self):
        return len(self.data)  # 데이터셋의 전체 샘플 수를 반환합니다.
    
    def __getitem__(self, idx):
        # 인덱스 `idx`에 해당하는 데이터 샘플을 반환합니다.
        
        # 데이터프레임에서 'survived' 열을 제외한 특성값을 가져와서 NumPy 배열로 변환한 뒤, PyTorch 텐서로 변환합니다.
        X = torch.from_numpy(self.data.drop('survived', axis=1).iloc[idx].values).float()
        
        # 'survived' 열의 값을 텐서로 변환하여 레이블을 생성합니다.
        y = torch.tensor(self.data.iloc[idx].survived).long()
        
        # 입력 데이터와 레이블을 딕셔너리 형태로 반환합니다.
        return {
            'X': X,
            'y': y,
        }

# 배치 크기 (Batch Size): 한 번의 업데이트에서 사용할 데이터 샘플 수
batch_size = 128

# 에포크 수 (Epochs): 전체 데이터셋을 몇 번 반복하여 학습할 것인지 지정
epochs = 20

# 학습률 (Learning Rate): 모델 파라미터를 업데이트할 때 사용되는 학습률
learning_rate = 1e-3

# 은닉층의 뉴런 수 (Hidden Dimension): 은닉층의 차원 또는 뉴런 수
hidden_dim = 64
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 시드를 설정하여 실험의 재현성을 보장합니다.
seed = 6
random.seed(seed)
np.random.seed(seed)  # NumPy의 시드를 설정합니다.
torch.manual_seed(seed)  # PyTorch의 시드를 설정합니다.

# 장치가 CUDA(GPU)일 경우, CUDA의 시드를 설정합니다.
if device == 'cuda':
    torch.cuda.manual_seed(seed)  # 현재 장치의 CUDA 시드를 설정합니다.
    torch.cuda.manual_seed_all(seed)  # 모든 CUDA 장치의 시드를 설정합니다.
    torch.backends.cudnn.deterministic = False  # Deterministic 연산을 비활성화합니다. 비활성화하면 더 빠른 연산이 가능할 수 있습니다.
    torch.backends.cudnn.benchmark = True  # 벤치마크를 활성화하여 최적의 성능을 위해 CUDA 커널을 자동으로 튜닝합니다.



C:\Users\USER\AppData\Local\Temp\ipykernel_16972\2111298443.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 2 0 2 0 2 2 1 2 0 2 2 2 1 2 2 1 1 2 0 2 2 0 0 1 0 0 2 2 2 2 1 1 2 2 2
 2 0 1 0 1 2 1 2 2 0 2 1 2 2 2 1 2 1 2 2 2 1 2 2 2 0 1 2 2 0 2 2 2 0 2 2 0
 0 1 1 2 0 2 2 2 2 2 0 2 2 2 2 2 2 1 0 2 1 1 1 0 2 2 2 2 2 2 1 1 1 0 0 2 0
 2 2 2 1 1 2 2 1 1 1 0 2 2 0 2 2 2 1 2 2 2 2 2 2 0 2 2 2 0 2 0 1 2 2 1 2 0
 2 2 1 1 2 1 0 0 2 1 2 2 2 2 2 2 2 2 0 2 1 2 1 0 2 1 0 1 2 1 2 0 2 1 2 1 0
 2 1 2 1 1 1 1 1 1 2 2 0 2 1 0 1 2 0 2 2 2 0 0 1 2 0 0 1 2 2 0 0 2 1 0 0 2
 2 2 2 2 2 2 2 2 2 1 2 0 0 1 2 2 2 0 0 2 0 0 1 0 0 0 1 2 1 2 1 1 0 0 2 2 1
 1 0 2 1 2 0 0 0 2 0 0 2 0 1 0 1 1 1 1 1 2 2 2 2 2 2 0 1 2 1 2 2 2 0 0 0 2
 2 0 2 2 0 2 2 0 2 2 0 1 2 1 1 0 2 2 0 2 2 2 1 1 1 2 2 2 2 2 1 2 1 2 0 2 1
 1 1 2 2 2 2 2 1 1 2 0 1 2 0 0 2 1 0 1 1 2 2 1 0 1 0 2 0 1 0 0 2 0 1 0 2 0
 1 2 0 2 2 1 1 2 1 2 2 2 2 2 2 0 0 0 2 2 2 0 0 2 0 0 2 2 2 2 0 0 

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 시드를 설정하여 실험의 재현성을 보장합니다.
seed = 6
random.seed(seed)
np.random.seed(seed)  # NumPy의 시드를 설정합니다.
torch.manual_seed(seed)  # PyTorch의 시드를 설정합니다.

# 장치가 CUDA(GPU)일 경우, CUDA의 시드를 설정합니다.
if device == 'cuda':
    torch.cuda.manual_seed(seed)  # 현재 장치의 CUDA 시드를 설정합니다.
    torch.cuda.manual_seed_all(seed)  # 모든 CUDA 장치의 시드를 설정합니다.
    torch.backends.cudnn.deterministic = False  # Deterministic 연산을 비활성화합니다. 비활성화하면 더 빠른 연산이 가능할 수 있습니다.
    torch.backends.cudnn.benchmark = True  # 벤치마크를 활성화하여 최적의 성능을 위해 CUDA 커널을 자동으로 튜닝합니다.
titanic = sns.load_dataset('titanic')
titanic.deck = titanic.deck.astype(str)
titanic = titanic.dropna()

titanic, _ = convert_category_into_integer(titanic, ('sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive'))

titanic = titanic.astype(np.float32)

train, temp = train_test_split(titanic, test_size=0.4, random_state=seed)

# 임시 데이터를 검증용 데이터와 테스트용 데이터로 분할
# 임시 데이터의 절반을 검증용 데이터로, 나머지 절반을 테스트용 데이터로 사용
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)



class TitanicDataset(Dataset):
    def __init__(self, data):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.data = data  # 데이터프레임을 저장합니다.
    
    def __len__(self):
        return len(self.data)  # 데이터셋의 전체 샘플 수를 반환합니다.
    
    def __getitem__(self, idx):
        # 인덱스 `idx`에 해당하는 데이터 샘플을 반환합니다.
        
        # 데이터프레임에서 'survived' 열을 제외한 특성값을 가져와서 NumPy 배열로 변환한 뒤, PyTorch 텐서로 변환합니다.
        X = torch.from_numpy(self.data.drop('survived', axis=1).iloc[idx].values).float()
        
        # 'survived' 열의 값을 텐서로 변환하여 레이블을 생성합니다.
        y = torch.tensor(self.data.iloc[idx].survived).long()
        
        # 입력 데이터와 레이블을 딕셔너리 형태로 반환합니다.
        return {
            'X': X,
            'y': y,
        }

# 배치 크기 (Batch Size): 한 번의 업데이트에서 사용할 데이터 샘플 수
batch_size = 128

# 에포크 수 (Epochs): 전체 데이터셋을 몇 번 반복하여 학습할 것인지 지정
epochs = 20

# 학습률 (Learning Rate): 모델 파라미터를 업데이트할 때 사용되는 학습률
learning_rate = 1e-3

# 은닉층의 뉴런 수 (Hidden Dimension): 은닉층의 차원 또는 뉴런 수
hidden_dim = 64

C:\Users\USER\AppData\Local\Temp\ipykernel_7160\1783900557.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 2 0 2 0 2 2 1 2 0 2 2 2 1 2 2 1 1 2 0 2 2 0 0 1 0 0 2 2 2 2 1 1 2 2 2
 2 0 1 0 1 2 1 2 2 0 2 1 2 2 2 1 2 1 2 2 2 1 2 2 2 0 1 2 2 0 2 2 2 0 2 2 0
 0 1 1 2 0 2 2 2 2 2 0 2 2 2 2 2 2 1 0 2 1 1 1 0 2 2 2 2 2 2 1 1 1 0 0 2 0
 2 2 2 1 1 2 2 1 1 1 0 2 2 0 2 2 2 1 2 2 2 2 2 2 0 2 2 2 0 2 0 1 2 2 1 2 0
 2 2 1 1 2 1 0 0 2 1 2 2 2 2 2 2 2 2 0 2 1 2 1 0 2 1 0 1 2 1 2 0 2 1 2 1 0
 2 1 2 1 1 1 1 1 1 2 2 0 2 1 0 1 2 0 2 2 2 0 0 1 2 0 0 1 2 2 0 0 2 1 0 0 2
 2 2 2 2 2 2 2 2 2 1 2 0 0 1 2 2 2 0 0 2 0 0 1 0 0 0 1 2 1 2 1 1 0 0 2 2 1
 1 0 2 1 2 0 0 0 2 0 0 2 0 1 0 1 1 1 1 1 2 2 2 2 2 2 0 1 2 1 2 2 2 0 0 0 2
 2 0 2 2 0 2 2 0 2 2 0 1 2 1 1 0 2 2 0 2 2 2 1 1 1 2 2 2 2 2 1 2 1 2 0 2 1
 1 1 2 2 2 2 2 1 1 2 0 1 2 0 0 2 1 0 1 1 2 2 1 0 1 0 2 0 1 0 0 2 0 1 0 2 0
 1 2 0 2 2 1 1 2 1 2 2 2 2 2 2 0 0 0 2 2 2 0 0 2 0 0 2 2 2 2 0 0 1

C:\Users\USER\AppData\Local\Temp\ipykernel_7160\1783900557.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 2 0 2 0 2 2 1 2 0 2 2 2 1 2 2 1 1 2 0 2 2 0 0 1 0 0 2 2 2 2 1 1 2 2 2
 2 0 1 0 1 2 1 2 2 0 2 1 2 2 2 1 2 1 2 2 2 1 2 2 2 0 1 2 2 0 2 2 2 0 2 2 0
 0 1 1 2 0 2 2 2 2 2 0 2 2 2 2 2 2 1 0 2 1 1 1 0 2 2 2 2 2 2 1 1 1 0 0 2 0
 2 2 2 1 1 2 2 1 1 1 0 2 2 0 2 2 2 1 2 2 2 2 2 2 0 2 2 2 0 2 0 1 2 2 1 2 0
 2 2 1 1 2 1 0 0 2 1 2 2 2 2 2 2 2 2 0 2 1 2 1 0 2 1 0 1 2 1 2 0 2 1 2 1 0
 2 1 2 1 1 1 1 1 1 2 2 0 2 1 0 1 2 0 2 2 2 0 0 1 2 0 0 1 2 2 0 0 2 1 0 0 2
 2 2 2 2 2 2 2 2 2 1 2 0 0 1 2 2 2 0 0 2 0 0 1 0 0 0 1 2 1 2 1 1 0 0 2 2 1
 1 0 2 1 2 0 0 0 2 0 0 2 0 1 0 1 1 1 1 1 2 2 2 2 2 2 0 1 2 1 2 2 2 0 0 0 2
 2 0 2 2 0 2 2 0 2 2 0 1 2 1 1 0 2 2 0 2 2 2 1 1 1 2 2 2 2 2 1 2 1 2 0 2 1
 1 1 2 2 2 2 2 1 1 2 0 1 2 0 0 2 1 0 1 1 2 2 1 0 1 0 2 0 1 0 0 2 0 1 0 2 0
 1 2 0 2 2 1 1 2 1 2 2 2 2 2 2 0 0 0 2 2 2 0 0 2 0 0 2 2 2 2 0 0 1

In [57]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 시드를 설정하여 실험의 재현성을 보장합니다.
seed = 0
random.seed(seed)
np.random.seed(seed)  # NumPy의 시드를 설정합니다.
torch.manual_seed(seed)  # PyTorch의 시드를 설정합니다.

# 장치가 CUDA(GPU)일 경우, CUDA의 시드를 설정합니다.
if device == 'cuda':
    torch.cuda.manual_seed(seed)  # 현재 장치의 CUDA 시드를 설정합니다.
    torch.cuda.manual_seed_all(seed)  # 모든 CUDA 장치의 시드를 설정합니다.
    torch.backends.cudnn.deterministic = False  # Deterministic 연산을 비활성화합니다. 비활성화하면 더 빠른 연산이 가능할 수 있습니다.
    torch.backends.cudnn.benchmark = True  # 벤치마크를 활성화하여 최적의 성능을 위해 CUDA 커널을 자동으로 튜닝합니다.


titanic = sns.load_dataset('titanic')
titanic.deck = titanic.deck.astype(str)
titanic = titanic.dropna()

titanic, _ = convert_category_into_integer(titanic, ('sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive'))

titanic = titanic.astype(np.float32)

train, temp = train_test_split(titanic, test_size=0.4, random_state=seed)

# 임시 데이터를 검증용 데이터와 테스트용 데이터로 분할
# 임시 데이터의 절반을 검증용 데이터로, 나머지 절반을 테스트용 데이터로 사용
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)



class TitanicDataset(Dataset):
    def __init__(self, data):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.data = data  # 데이터프레임을 저장합니다.
    
    def __len__(self):
        return len(self.data)  # 데이터셋의 전체 샘플 수를 반환합니다.
    
    def __getitem__(self, idx):
        # 인덱스 `idx`에 해당하는 데이터 샘플을 반환합니다.
        
        # 데이터프레임에서 'survived' 열을 제외한 특성값을 가져와서 NumPy 배열로 변환한 뒤, PyTorch 텐서로 변환합니다.
        X = torch.from_numpy(self.data.drop('survived', axis=1).iloc[idx].values).float()
        
        # 'survived' 열의 값을 텐서로 변환하여 레이블을 생성합니다.
        y = torch.tensor(self.data.iloc[idx].survived).long()
        
        # 입력 데이터와 레이블을 딕셔너리 형태로 반환합니다.
        return {
            'X': X,
            'y': y,
        }

# 배치 크기 (Batch Size): 한 번의 업데이트에서 사용할 데이터 샘플 수
batch_size = 64

# 에포크 수 (Epochs): 전체 데이터셋을 몇 번 반복하여 학습할 것인지 지정
epochs = 1000

# 학습률 (Learning Rate): 모델 파라미터를 업데이트할 때 사용되는 학습률
learning_rate = 1e-3

# 은닉층의 뉴런 수 (Hidden Dimension): 은닉층의 차원 또는 뉴런 수
hidden_dim = 64



# 학습 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = TitanicDataset(train)

# 검증 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
valid_dataset = TitanicDataset(valid)

# 테스트 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
test_dataset = TitanicDataset(test)




class Model(nn.Module):  # nn.Module을 상속받아 새로운 모델 클래스를 정의합니다.
    def __init__(self, input_dim, hidden_dim, output_dim):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.input_dim = input_dim  # 입력 차원 크기를 저장합니다.
        self.hidden_dim = hidden_dim  # 숨겨진 층의 차원 크기를 저장합니다.
        self.output_dim = output_dim  # 출력 차원 크기를 저장합니다.

        self.linear = nn.Linear(input_dim, hidden_dim)  # 입력 차원에서 숨겨진 차원으로의 선형 변환을 정의합니다.
        self.relu = nn.ReLU()  # ReLU 활성화 함수를 정의합니다.
        self.output = nn.Linear(hidden_dim, output_dim)  # 숨겨진 차원에서 출력 차원으로의 선형 변환을 정의합니다.
    
    def forward(self, x):  # 순전파 메서드
        x = self.linear(x)  # 입력 데이터에 대해 선형 변환을 적용합니다.
        x = self.relu(x)  # ReLU 활성화 함수를 적용하여 비선형성을 추가합니다.
        x = self.output(x)  # 두 번째 선형 변환을 적용하여 최종 출력을 계산합니다.

        return x  # 최종 출력을 반환합니다.
    
# len(titanic.columns) - 1: 입력 특성의 수 (타이타닉 데이터셋에서 'survived' 열을 제외한 나머지 열)
# hidden_dim: 은닉층의 뉴런 수 (변수로 설정된 값)
# 2: 출력 클래스의 수 (이진 분류 문제이므로 두 개 클래스)
model = Model(len(titanic.columns) - 1, hidden_dim, 2)

class TitanicDataModule(L.LightningDataModule):
    def __init__(
        self,
        batch_size: int,
        ):
        super().__init__()
        self.batch_size = batch_size

    def prepare(self, train_dataset, valid_dataset, test_dataset):     
        self.train_dataset, self.valid_dataset, self.test_dataset = train_dataset, valid_dataset, test_dataset

    def setup(self, stage: str):
        if stage == "fit":      
            self.train_data = self.train_dataset
            self.valid_data = self.valid_dataset

        if stage == "test":     
            self.test_data = self.test_dataset

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train_data,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.valid_data,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test_data,
            batch_size=self.batch_size,
            shuffle=False,
        )
    
titanic_data_module = TitanicDataModule(batch_size=batch_size)
titanic_data_module.prepare(train_dataset, valid_dataset, test_dataset)

class TitanicModule(L.LightningModule):
    def __init__(
        self,
        model: torch.nn.Module,    # 구축한 모델
        learning_rate: float,      # 학습률
    ):
        super().__init__()
        self.model = model         # 모델 초기화
        self.learning_rate = learning_rate  # 학습률 초기화
    
    def training_step(self, batch):

        X = batch.get('X')  # 입력 데이터를 장치로 이동
        y = batch.get('y')  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        self.loss = F.cross_entropy(logit, y)  # 손실 계산

        predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산
        self.acc = (predicted_label == y).float().mean()  # 정확도 계산


        return self.loss  # 손실 반환
    
    def on_train_epoch_end(self, *args, **kwargs):
        # 학습 에포크가 끝날 때 호출되는 메서드
        self.log_dict({'acc/train_acc': self.acc, 'loss/train_loss': self.loss}, on_epoch=True, prog_bar=True, logger=True)
    
    def validation_step(self, batch):

        X = batch.get('X')  # 입력 데이터를 장치로 이동
        y = batch.get('y')  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        self.val_loss = F.cross_entropy(logit, y)  # 검증 손실 계산

        predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산
        self.val_acc = (predicted_label == y).float().mean()  # 정확도 계산


        return self.val_loss  # 검증 손실 반환
    
    def on_validation_epoch_end(self):
        # 검증 에포크가 끝날 때 호출되는 메서드
        self.log_dict({'acc/val_acc': self.val_acc, 'loss/val_loss': self.val_loss}, on_epoch=True, prog_bar=True, logger=True)
    
    def test_step(self, batch, batch_idx):
        # 테스트 단계에서 호출되는 메서드
        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산

        return predicted_label  # 예측된 레이블 반환

    def configure_optimizers(self):
        # 옵티마이저를 설정하는 메서드
        optimizer = optim.Adam(
            self.model.parameters(),  # 모델 파라미터를 옵티마이저에 전달
            lr=self.learning_rate,    # 학습률 설정
        )

        return {'optimizer': optimizer}  # 옵티마이저 반환
    
# TitanicModule 클래스의 인스턴스를 생성합니다.
titanic_module = TitanicModule(
    model=model,              # 학습할 모델 인스턴스 (예: Model 객체)
    learning_rate=learning_rate  # 학습률 (예: 1e-3)
)
# Trainer 인스턴스 생성
trainer = Trainer(
    max_epochs=epochs,  # 학습할 최대 에포크 수
    callbacks=[
        EarlyStopping(monitor='loss/val_loss', mode='min', patience= 5)
    ],
    logger= TensorBoardLogger(
        "tensorboard",                                                # root folder
        f"seed= {seed}, batch_size= {batch_size}, learning_rate= {learning_rate}, hidden_dim= {hidden_dim}",   # folder name
    ),
)
# PyTorch Lightning Trainer를 사용하여 모델 학습을 시작
# 'titanic_module'은 학습할 모델과 학습 및 검증 루프를 정의한 LightningModule 인스턴스
# 'titanic_data_module'은 데이터셋을 로드하고 데이터로더를 제공하는 LightningDataModule 인스턴스
trainer.fit(
    model=titanic_module,       # 학습할 모델 인스턴스
    datamodule=titanic_data_module,  # 데이터셋과 데이터로더를 제공하는 데이터 모듈 인스턴스
)

C:\Users\USER\AppData\Local\Temp\ipykernel_7160\1783900557.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 2 0 2 0 2 2 1 2 0 2 2 2 1 2 2 1 1 2 0 2 2 0 0 1 0 0 2 2 2 2 1 1 2 2 2
 2 0 1 0 1 2 1 2 2 0 2 1 2 2 2 1 2 1 2 2 2 1 2 2 2 0 1 2 2 0 2 2 2 0 2 2 0
 0 1 1 2 0 2 2 2 2 2 0 2 2 2 2 2 2 1 0 2 1 1 1 0 2 2 2 2 2 2 1 1 1 0 0 2 0
 2 2 2 1 1 2 2 1 1 1 0 2 2 0 2 2 2 1 2 2 2 2 2 2 0 2 2 2 0 2 0 1 2 2 1 2 0
 2 2 1 1 2 1 0 0 2 1 2 2 2 2 2 2 2 2 0 2 1 2 1 0 2 1 0 1 2 1 2 0 2 1 2 1 0
 2 1 2 1 1 1 1 1 1 2 2 0 2 1 0 1 2 0 2 2 2 0 0 1 2 0 0 1 2 2 0 0 2 1 0 0 2
 2 2 2 2 2 2 2 2 2 1 2 0 0 1 2 2 2 0 0 2 0 0 1 0 0 0 1 2 1 2 1 1 0 0 2 2 1
 1 0 2 1 2 0 0 0 2 0 0 2 0 1 0 1 1 1 1 1 2 2 2 2 2 2 0 1 2 1 2 2 2 0 0 0 2
 2 0 2 2 0 2 2 0 2 2 0 1 2 1 1 0 2 2 0 2 2 2 1 1 1 2 2 2 2 2 1 2 1 2 0 2 1
 1 1 2 2 2 2 2 1 1 2 0 1 2 0 0 2 1 0 1 1 2 2 1 0 1 0 2 0 1 0 0 2 0 1 0 2 0
 1 2 0 2 2 1 1 2 1 2 2 2 2 2 2 0 0 0 2 2 2 0 0 2 0 0 2 2 2 2 0 0 1

c:\Users\USER\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\USER\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\USER\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 941:  43%|████▎     | 3/7 [00:00<00:00, 47.53it/s, v_num=3, acc/val_acc=1.000, loss/val_loss=0.313, acc/train_acc=1.000, loss/train_loss=0.313] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [66]:
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53789,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53790,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53791,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53792,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [69]:

diamonds = sns.load_dataset('diamonds')
diamonds = diamonds.drop_duplicates().reset_index(drop=True)
diamonds, _ = convert_category_into_integer(diamonds, ('cut', 'color', 'clarity'))

diamonds
standard_scaler = StandardScaler()

train.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.fit_transform(train.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

# 검증 세트의 동일한 열을 훈련 세트에서 계산된 평균과 표준편차를 사용하여 표준화합니다.
valid.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.transform(valid.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

# 테스트 세트의 동일한 열을 훈련 세트에서 계산된 평균과 표준편차를 사용하여 표준화합니다.
test.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.transform(test.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

C:\Users\USER\AppData\Local\Temp\ipykernel_7160\1783900557.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_7160\1783900557.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_7160\1783900557.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast to a compatible dtype firs

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53789,0.72,2,0,2,60.8,57.0,2757,5.75,5.76,3.50
53790,0.72,1,0,2,63.1,55.0,2757,5.69,5.75,3.61
53791,0.70,4,0,2,62.8,60.0,2757,5.66,5.68,3.56
53792,0.86,3,4,3,61.0,58.0,2757,6.15,6.12,3.74


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 시드를 설정하여 실험의 재현성을 보장합니다.
seed = 0
random.seed(seed)
np.random.seed(seed)  # NumPy의 시드를 설정합니다.
torch.manual_seed(seed)  # PyTorch의 시드를 설정합니다.

# 장치가 CUDA(GPU)일 경우, CUDA의 시드를 설정합니다.
if device == 'cuda':
    torch.cuda.manual_seed(seed)  # 현재 장치의 CUDA 시드를 설정합니다.
    torch.cuda.manual_seed_all(seed)  # 모든 CUDA 장치의 시드를 설정합니다.
    torch.backends.cudnn.deterministic = False  # Deterministic 연산을 비활성화합니다. 비활성화하면 더 빠른 연산이 가능할 수 있습니다.
    torch.backends.cudnn.benchmark = True  # 벤치마크를 활성화하여 최적의 성능을 위해 CUDA 커널을 자동으로 튜닝합니다.


diamonds = sns.load_dataset('diamonds')
diamonds = diamonds.drop_duplicates().reset_index(drop=True)
diamonds, _ = convert_category_into_integer(diamonds, ('cut', 'color', 'clarity'))

train, temp = train_test_split(diamonds, test_size=0.4, random_state=seed)

# 임시 데이터를 검증용 데이터와 테스트용 데이터로 분할
# 임시 데이터의 절반을 검증용 데이터로, 나머지 절반을 테스트용 데이터로 사용
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

standard_scaler = StandardScaler()

train.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.fit_transform(train.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

# 검증 세트의 동일한 열을 훈련 세트에서 계산된 평균과 표준편차를 사용하여 표준화합니다.
valid.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.transform(valid.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

# 테스트 세트의 동일한 열을 훈련 세트에서 계산된 평균과 표준편차를 사용하여 표준화합니다.
test.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.transform(test.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )



class DiamondsDataset(Dataset):
    def __init__(self, data):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.data = data  # 데이터프레임을 저장합니다.
    
    def __len__(self):
        return len(self.data)  # 데이터셋의 전체 샘플 수를 반환합니다.
    
    def __getitem__(self, idx):
        # 인덱스 `idx`에 해당하는 데이터 샘플을 반환합니다.
        
        # 데이터프레임에서 'price' 열을 제외한 특성값을 가져와서 NumPy 배열로 변환한 뒤, PyTorch 텐서로 변환합니다.
        X = torch.from_numpy(self.data.iloc[idx].drop('price').values).float()
        
        # 'price' 열의 값을 텐서로 변환하여 레이블을 생성합니다.
        y = torch.Tensor([self.data.iloc[idx].price]).float()
        
        # 입력 데이터와 레이블을 딕셔너리 형태로 반환합니다.
        return {
            'X': X,
            'y': y,
        }

# 배치 크기 (Batch Size): 한 번의 업데이트에서 사용할 데이터 샘플 수
batch_size = 1024  # 데이터 배치의 크기를 설정합니다.
epochs = 100  # 학습 에포크 수를 설정합니다.
learning_rate = 1e-1  # 학습률을 설정합니다.
hidden_dim = 32  # 숨겨진 층의 차원 크기를 설정합니다.
dropout_prob = 0.5



# 학습 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = DiamondsDataset(train)

# 검증 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
valid_dataset = DiamondsDataset(valid)

# 테스트 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
test_dataset = DiamondsDataset(test)




class Model(nn.Module):  # nn.Module을 상속받아 새로운 모델 클래스를 정의합니다.
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.input_dim = input_dim  # 입력 차원 크기를 저장합니다.
        self.hidden_dim = hidden_dim  # 숨겨진 층의 차원 크기를 저장합니다.
        self.output_dim = output_dim  # 출력 차원 크기를 저장합니다.
        self.dropout_prob =dropout_prob

        self.linear = nn.Linear(input_dim, hidden_dim)  # 입력 차원에서 숨겨진 차원으로의 선형 변환을 정의합니다.
        self.relu = nn.ReLU()  # ReLU 활성화 함수를 정의합니다.
        self.dropout1 = nn.Dropout(self.dropout_prob)
        self.output = nn.Linear(hidden_dim, output_dim)  # 숨겨진 차원에서 출력 차원으로의 선형 변환을 정의합니다.
    
    def forward(self, x):  # 순전파 메서드
        x = self.linear(x)  # 입력 데이터에 대해 선형 변환을 적용합니다.
        x = self.relu(x)  # ReLU 활성화 함수를 적용하여 비선형성을 추가합니다.
        x = self.dropout1(x)
        x = self.output(x)  # 두 번째 선형 변환을 적용하여 최종 출력을 계산합니다.

        return x  # 최종 출력을 반환합니다.
    
# len(titanic.columns) - 1: 입력 특성의 수 (타이타닉 데이터셋에서 'survived' 열을 제외한 나머지 열)
# hidden_dim: 은닉층의 뉴런 수 (변수로 설정된 값)
# 2: 출력 클래스의 수 (이진 분류 문제이므로 두 개 클래스)
# Model 클래스를 사용하여 모델 인스턴스를 생성합니다.
model = Model(
    input_dim=len(diamonds.columns)-1,  # 입력 차원 크기를 설정합니다. diamonds 데이터프레임의 열 개수에서 1을 뺀 값입니다.
    hidden_dim=hidden_dim,  # 숨겨진 층의 차원 크기를 설정합니다. 이 값은 미리 정의된 `hidden_dim` 변수로부터 가져옵니다.
    output_dim=1,  # 출력 차원 크기를 설정합니다. 'cut' 열의 고유한 값의 개수를 가져옵니다.
    dropout_prob =dropout_prob,
).to(device)

class DiamondsDataModule(L.LightningDataModule):
    def __init__(
        self,
        batch_size: int,
        ):
        super().__init__()
        self.batch_size = batch_size

    def prepare(self, train_dataset, valid_dataset, test_dataset):     
        self.train_dataset, self.valid_dataset, self.test_dataset = train_dataset, valid_dataset, test_dataset

    def setup(self, stage: str):
        if stage == "fit":      
            self.train_data = self.train_dataset
            self.valid_data = self.valid_dataset

        if stage == "test":     
            self.test_data = self.test_dataset

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train_data,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.valid_data,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test_data,
            batch_size=self.batch_size,
            shuffle=False,
        )
    
diamonds_data_module = DiamondsDataModule(batch_size=batch_size)
diamonds_data_module.prepare(train_dataset, valid_dataset, test_dataset)

class DiamondsModule(L.LightningModule):
    def __init__(
        self,
        model: torch.nn.Module,    # 구축한 모델
        learning_rate: float,      # 학습률
    ):
        super().__init__()
        self.model = model         # 모델 초기화
        self.learning_rate = learning_rate  # 학습률 초기화
    
    def training_step(self, batch):

        X = batch.get('X')  # 입력 데이터를 장치로 이동
        y = batch.get('y')  # 레이블 데이터를 장치로 이동
         # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        #logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        self.loss = F.mse_loss(output, y)  # 손실 계산

        # predicted_label = self.loss.argmax(dim=-1)  # 예측된 레이블 계산
        # self.acc = (predicted_label == y).float().mean()  # 정확도 계산


        return self.loss  # 손실 반환
    
    def on_train_epoch_end(self, *args, **kwargs):
        # 학습 에포크가 끝날 때 호출되는 메서드
        self.log_dict({'loss/train_loss': self.loss}, on_epoch=True, prog_bar=True, logger=True)

       
    
    def validation_step(self, batch):

        X = batch.get('X')  # 입력 데이터를 장치로 이동
        y = batch.get('y')  # 레이블 데이터를 장치로 이동  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        # logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        self.val_loss = F.mse_loss(output, y)  # 검증 손실 계산

        predicted_label = self.val_loss.argmax(dim=-1)  # 예측된 레이블 계산
        self.val_acc = (predicted_label == y).float().mean()  # 정확도 계산


        return self.val_loss  # 검증 손실 반환
    
    def on_validation_epoch_end(self):
        # 검증 에포크가 끝날 때 호출되는 메서드
        self.log_dict({'loss/val_loss': self.val_loss}, on_epoch=True, prog_bar=True, logger=True)
        self.log_dict({'learning_rate': self.learning_rate}, on_epoch=True, prog_bar=True, logger=True)
    
    def test_step(self, batch, batch_idx):
        # 테스트 단계에서 호출되는 메서드
        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
    

        return output  # 예측된 레이블 반환

    def configure_optimizers(self):
        # 옵티마이저를 설정하는 메서드
        optimizer = optim.Adam(
            self.model.parameters(),  # 모델 파라미터를 옵티마이저에 전달
            lr=self.learning_rate,    # 학습률 설정
        )

        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode= 'min',
            factor= 0.5,
            patience= 5,
        )
   
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }  # 옵티마이저 반환
    
# TitanicModule 클래스의 인스턴스를 생성합니다.
diamonds_module = DiamondsModule(
    model=model,              # 학습할 모델 인스턴스 (예: Model 객체)
    learning_rate=learning_rate  # 학습률 (예: 1e-3)
)
# Trainer 인스턴스 생성
trainer = Trainer(
    max_epochs=epochs,  # 학습할 최대 에포크 수
    callbacks=[
        EarlyStopping(monitor='loss/val_loss', mode='min', patience= 5)
    ],
    logger= TensorBoardLogger(
        "tensorboard",                                                # root folder
        f"seed= {seed}, batch_size= {batch_size}, learning_rate= {learning_rate}, hidden_dim= {hidden_dim}, dropout_prob={dropout_prob}",   # folder name
    ),
)
# PyTorch Lightning Trainer를 사용하여 모델 학습을 시작
# 'titanic_module'은 학습할 모델과 학습 및 검증 루프를 정의한 LightningModule 인스턴스
# 'titanic_data_module'은 데이터셋을 로드하고 데이터로더를 제공하는 LightningDataModule 인스턴스
trainer.fit(
    model=diamonds_module,       # 학습할 모델 인스턴스
    datamodule=diamonds_data_module,  # 데이터셋과 데이터로더를 제공하는 데이터 모듈 인스턴스
)

C:\Users\USER\AppData\Local\Temp\ipykernel_16972\2111298443.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_16972\2111298443.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:, column] = label_encoder.fit_transform(df[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_16972\2111298443.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast to a compatible dtype f

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\USER\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\USER\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\USER\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (32) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 14: 100%|██████████| 32/32 [00:09<00:00,  3.38it/s, v_num=0, loss/val_loss=0.127, learning_rate=0.100, loss/train_loss=0.264] 


In [ ]:
> class Model(nn.Module):
>     def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
>         super().__init__()
>         self.input_dim = input_dim
>         self.hidden_dim = hidden_dim
>         self.output_dim = output_dim
> 
>         self.linear1 = nn.Linear(input_dim, hidden_dim)
>         self.relu1 = nn.ReLU()
>         self.dropout1 = nn.Dropout(dropout_prob)  # 첫 번째 드롭아웃
>         self.linear2 = nn.Linear(hidden_dim, hidden_dim)
>         self.relu2 = nn.ReLU()
>         self.dropout2 = nn.Dropout(dropout_prob)  # 두 번째 드롭아웃
>         self.output = nn.Linear(hidden_dim, output_dim)
>     
>     def forward(self, x):
>         x = self.linear1(x)
>         x = self.relu1(x)
>         x = self.dropout1(x)  # 첫 번째 드롭아웃 적용
>         x = self.linear2(x)
>         x = self.relu2(x)
>         x = self.dropout2(x)  # 두 번째 드롭아웃 적용
>         x = self.output(x)
> 
>         return x
> ```